In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import nest_asyncio
nest_asyncio.apply()


In [3]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    PromptTemplate,
)
from IPython.display import Markdown, display


In [ ]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")



In [5]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

In [ ]:
from llama_index.core import VectorStoreIndex

# vector_index = VectorStoreIndex.from_documents(documents, embed_model=ollama_embedding)
vector_index = VectorStoreIndex.from_documents(documents)


In [7]:
qa_prompt_tmpl_str = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Please write the answer in the style of {tone_name}
Query: {query_str}
Answer: \
"""

In [12]:
# Create a PromptTemplate object
prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

# Apply partial formatting to set the tone_name
partial_prompt_tmpl = prompt_tmpl.partial_format(tone_name="Shakespeare")

In [17]:
formatted_prompt = partial_prompt_tmpl.format(
    context_str="In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters",
    query_str="How many params does Llama 2 have?"
)

In [ ]:
response = vector_index.as_query_engine(
    llm=ollama_llm
).query(formatted_prompt)


In [ ]:
print("\nResponse from the Query Engine:")
print(response)